In [1]:
import math
import numpy as np
import torch
from datasets import load_dataset
from transformers import RobertaForSequenceClassification, RobertaTokenizer, RobertaTokenizerFast, RobertaConfig, RobertaModelWithHeads
from transformers import Trainer, TrainingArguments, EvalPrediction
from transformers import DataCollatorForLanguageModeling
from transformers import RobertaForMaskedLM
from transformers import AdapterType
from sklearn.metrics import f1_score

In [2]:
import datetime

In [3]:
today = datetime.date.today().strftime('%Y%m%d')

In [4]:
def check_gpu():
    '''
    check gpu status
    '''
    try:
        print('GPU available:', torch.cuda.is_available())
        print(torch.cuda.device_count(), 'GPUs detected')
        print('Current GPU id:', torch.cuda.current_device())
        print('Current GPU Name:', torch.cuda.get_device_name(torch.cuda.current_device()))
    except:
        print('GPU not available')

In [5]:
def encode_batch(batch):
    '''
    Encodes a batch of input data using the model tokenizer
    '''
    return tokenizer(batch["text"], max_length=80, truncation=True, padding="max_length")
#     return tokenizer(batch["text"], padding=True, truncation=True)

In [6]:
check_gpu()

GPU available: True
1 GPUs detected
Current GPU id: 0
Current GPU Name: NVIDIA GeForce GTX 980 Ti


In [7]:
# dictionary for dataset, name: (classes, type of f1 score)
dataset_dict = {'chemprot': (13, 'micro'), 'rct': (5, 'micro'),
                'CI': (6, 'macro'), 'sciie': (7, 'm2cro'),
                'HN': (2, 'macro'), 'ag': (4, 'macro'),
                'amazon': (2, 'macro'), 'imdb': (2, 'macro')}

In [8]:
ds_name = 'amazon'
n_labels = dataset_dict[ds_name][0]
f1_type = dataset_dict[ds_name][1]

In [9]:
dataset = load_dataset(f'data_loaders/{ds_name}_data_loader.py')

Reusing dataset task_dataset (C:\Users\snow-\.cache\huggingface\datasets\task_dataset\task\1.0.0\d4dbb1ae1e5b21302597f18c62e58ab7f320999e2bdffea6d0514c3c329ad9ae)


In [10]:
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

In [11]:
#dataset_encoded = dataset.map(encode_batch, batched=True, batch_size=512, remove_columns=["text"])
dataset_encoded = dataset.map(encode_batch, batched=True)
# tokenized_datasets = dataset.map(tokenizer, batched=True, num_proc=4, remove_columns=["text"])

Loading cached processed dataset at C:\Users\snow-\.cache\huggingface\datasets\task_dataset\task\1.0.0\d4dbb1ae1e5b21302597f18c62e58ab7f320999e2bdffea6d0514c3c329ad9ae\cache-a7bd2fe3fa422dc1.arrow
Loading cached processed dataset at C:\Users\snow-\.cache\huggingface\datasets\task_dataset\task\1.0.0\d4dbb1ae1e5b21302597f18c62e58ab7f320999e2bdffea6d0514c3c329ad9ae\cache-1ed36f07475b7f6a.arrow
Loading cached processed dataset at C:\Users\snow-\.cache\huggingface\datasets\task_dataset\task\1.0.0\d4dbb1ae1e5b21302597f18c62e58ab7f320999e2bdffea6d0514c3c329ad9ae\cache-a74e264e5d1d56f8.arrow


In [12]:
dataset_encoded.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

## Case 2
fine tune a sequence classification model with a task corpus pretrained mlm model by adding a classification head

In [11]:
# load pretrained model
model = RobertaModelWithHeads.from_pretrained('model/amazon_mlm.pt', return_dict=True)

Some weights of the model checkpoint at model/amazon_mlm.pt were not used when initializing RobertaModelWithHeads: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at model/amazon_mlm.pt and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a d

In [14]:
# model.add_adapter("classifier", adapter_type=AdapterType.text_task, config="pfeiffer")

In [15]:
# model.train_adapter(['classifier'])

In [16]:
model.add_classification_head("classifier", num_labels=n_labels)

In [17]:
# model.set_active_adapters([["classifier"]])

In [18]:
# test model output
# model(**tokenizer("Hello, my dog is cute", return_tensors="pt"))
# test model output
# model(**tokenizer("Hello, my dog is cute", return_tensors="pt").to('cuda'))

In [19]:
# model.add_adapter('language', AdapterType.text_lang)

In [20]:
# model.train_adapter('language')

In [21]:
# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [22]:
training_args = TrainingArguments(
    output_dir=f'model/{ds_name}/ft-mlm/{today}/',
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    remove_unused_columns=False,
    num_train_epochs=2,
    fp16=True,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64
)

In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_encoded["train"],
    eval_dataset=dataset_encoded["validation"]
)

In [24]:
training_args

TrainingArguments(output_dir='model/amazon/ft-mlm/20210427/', overwrite_output_dir=False, do_train=False, do_eval=None, do_predict=False, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.EPOCH: 'epoch'>, prediction_loss_only=False, per_device_train_batch_size=32, per_device_eval_batch_size=64, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=2e-05, weight_decay=0.01, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=2, max_steps=-1, warmup_steps=0, logging_dir='runs\\Apr27_10-43-12_DESKTOP-HAMNFK0', logging_first_step=False, logging_steps=500, save_steps=500, save_total_limit=None, no_cuda=False, seed=42, fp16=True, fp16_opt_level='O1', local_rank=-1, tpu_num_cores=None, tpu_metrics_debug=False, debug=False, dataloader_drop_last=False, eval_steps=500, dataloader_num_workers=0, past_index=-1, run_name='model/amazon/ft-mlm/20210427/', disab

In [25]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.309073,0.311008
2,0.271893,0.319173


TrainOutput(global_step=7204, training_loss=0.30229963930628817)

In [26]:
trainer.evaluate()
# {'eval_loss': 0.31917300820350647, 'epoch': 2.0}

{'eval_loss': 0.31917300820350647, 'epoch': 2.0}

## Case 3
Compare with raw RoBerta sequence classification model

In [14]:
model_class = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=n_labels)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [15]:
training_args = TrainingArguments(
    output_dir=f'model/{ds_name}/ft-raw/{today}/',
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    remove_unused_columns=False,
    num_train_epochs=2,
    fp16=True,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64
)

In [16]:
trainer_class = Trainer(
    model=model_class,
    args=training_args,
    train_dataset=dataset_encoded["train"],
    eval_dataset=dataset_encoded["validation"]
)

In [17]:
trainer_class.train()

Epoch,Training Loss,Validation Loss
1,0.317709,0.316337
2,0.280312,0.321011


TrainOutput(global_step=7204, training_loss=0.3124534695893774)

In [18]:
trainer_class.evaluate()
# {'eval_loss': 0.32101142406463623, 'epoch': 2.0}

{'eval_loss': 0.32101142406463623, 'epoch': 2.0}

In [ ]:
training_args_dspt = TrainingArguments(
    output_dir=f'model/{ds_name}/ft-mlm/{today}/',
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    remove_unused_columns=False,
    num_train_epochs=2,
    fp16=True,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    adam_beta1=0.9,
    adam_beta2=0.98,
    adam_epsilon=1e-6,
    warmup_ratio=0.006,
    weight_decay=0.01,
    
)

In [ ]:
Hyperparameter Assignment
number of epochs 3 or 10
patience 3
batch size 16
learning rate 2e-5
dropout 0.1
feedforward layer 1
feedforward nonlinearity tanh
classification layer 1


## Case 4
adapter language model

In [11]:
# model = RobertaModelWithHeads.from_pretrained('roberta-base')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infere

In [13]:
model = RobertaForMaskedLM.from_pretrained('roberta-base')

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
model.add_adapter('mlm', AdapterType.text_lang)

In [15]:
model.train_adapter(["mlm"])

In [16]:
model.set_active_adapters(['mlm'])

In [27]:
# model.add_classification_head("classifier", num_labels=n_labels)

In [29]:
# model.add_adapter('classifier_adapter', AdapterType.text_task)

In [17]:
# get warm up steps for given warmup ratio

warmup_ratio=0.006
train_batch_size=32
dataset_encoded['train'].num_rows / train_batch_size * warmup_ratio

21.6095625

In [18]:
training_args = TrainingArguments(
    output_dir=f'model/{ds_name}/mlm-adapter/{today}/',
    evaluation_strategy = "epoch",
    learning_rate=0.00025,
    remove_unused_columns=False,
    num_train_epochs=2,
    fp16=True,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    adam_beta1=0.9,
    adam_beta2=0.98,
    adam_epsilon=1e-6,
#     warmup_ratio=0.006, not supported in adapter-transformers
    warmup_steps=21,
    weight_decay=0.01,
)

In [19]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_encoded["train"],
    eval_dataset=dataset_encoded["validation"],
    data_collator=data_collator,
)

In [21]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.716345,1.586855
2,1.670217,1.574642


TrainOutput(global_step=7204, training_loss=1.7298575173080928)

In [22]:
trainer.evaluate()

{'eval_loss': 1.5649161338806152, 'epoch': 2.0}

In [23]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 4.86


In [26]:
model.save_pretrained(f'model/amazon/{today}/model')

In [27]:
model.save_adapter(f'model/amazon/{today}/adapter', 'mlm')

In [28]:
f'model/amazon/{today}/model'

'model/amazon/20210427/model'

In [30]:
f'model/amazon/{today}/adapter'

'model/amazon/20210427/adapter'

## Case 5
task adapter after mlm adapter

In [13]:
# load model
model = RobertaModelWithHeads.from_pretrained('model/amazon/20210427/model')

Some weights of the model checkpoint at model/amazon/20210427/model were not used when initializing RobertaModelWithHeads: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at model/amazon/20210427/model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN t

In [14]:
model.roberta.encoder.layer[10].output.layer_text_lang_adapters

ModuleDict(
  (mlm): Adapter(
    (non_linearity): Activation_Function_Class()
    (adapter_down): Sequential(
      (0): Linear(in_features=768, out_features=48, bias=True)
      (1): Activation_Function_Class()
    )
    (adapter_up): Linear(in_features=48, out_features=768, bias=True)
  )
)

In [16]:
model.load_adapter(f'model/amazon/{today}/adapter')

Overwriting existing adapter 'mlm'.


'mlm'

In [17]:
model.add_classification_head("classifier", num_labels=n_labels)

In [18]:
model.add_adapter('amazon_classifer', AdapterType.text_task)

In [19]:
model.roberta.encoder.layer[10].output.layer_text_lang_adapters

ModuleDict(
  (mlm): Adapter(
    (non_linearity): Activation_Function_Class()
    (adapter_down): Sequential(
      (0): Linear(in_features=768, out_features=48, bias=True)
      (1): Activation_Function_Class()
    )
    (adapter_up): Linear(in_features=48, out_features=768, bias=True)
  )
)

In [20]:
model.roberta.encoder.layer[10].output.layer_text_task_adapters

ModuleDict(
  (amazon_classifer): Adapter(
    (non_linearity): Activation_Function_Class()
    (adapter_down): Sequential(
      (0): Linear(in_features=768, out_features=48, bias=True)
      (1): Activation_Function_Class()
    )
    (adapter_up): Linear(in_features=48, out_features=768, bias=True)
  )
)

In [21]:
model.train_adapter(["amazon_classifer"])

In [22]:
training_args = TrainingArguments(
    output_dir=f'model/{ds_name}/mlm-adapter/{today}/',
    evaluation_strategy = "epoch",
    learning_rate=0.00025,
    remove_unused_columns=False,
    num_train_epochs=2,
    fp16=True,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    adam_beta1=0.9,
    adam_beta2=0.98,
    adam_epsilon=1e-6,
#     warmup_ratio=0.006, not supported in adapter-transformers
    warmup_steps=21,
    weight_decay=0.01,
)

In [23]:
def compute_accuracy(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return {"acc": (preds == p.label_ids).mean()}

In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_accuracy
)

In [25]:
trainer.train()

ValueError: You have to specify either input_ids or inputs_embeds